# DerivaML Dataset Example.

DerivaML is a class library built on the Deriva Scientific Asset management system that is designed to help simplify a number of the basic operations associated with building and testing ML libraries based on common toolkits such as TensorFlow.  This notebook reviews the basic features of the DerivaML library.

## Set up DerivaML  for test case

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from demo_catalog import create_demo_catalog, DemoML
from deriva_ml import MLVocab as vt
from deriva_ml import DatasetBag
import pandas as pd
from IPython.display import display, Markdown, HTML

Set the details for the catalog we want and authenticate to the server if needed.

In [7]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


You are already logged in.


Create a test catalog and get an instance of the DemoML class.

In [8]:
test_catalog = create_demo_catalog(hostname, domain_schema)
ml_instance = DemoML(hostname, test_catalog.catalog_id)

## Configure DerivaML Datasets

In Deriva-ML a dataset is used to aggregate instances of entities.  However, before we can create any datasets, we must configure 
Deriva-ML for the specifics of the datasets.  The first stp is we need to tell Deriva-ML what types of use defined objects can be associated with a dataset.  

Note that out of the box, Deriva-ML is configured to allow datasets to contained dataset (i.e. nested datasets), so we don't need to do anything for that specific configuration.

In [9]:
print(f"Current dataset element types: {[a.name for a in ml_instance.list_dataset_element_types()]}")
ml_instance.add_dataset_element_type("Subject")
ml_instance.add_dataset_element_type("Image")
print(f"New dataset element types {[a.name for a in ml_instance.list_dataset_element_types()]}")

Current dataset element types: ['Dataset']
New dataset element types ['Dataset', 'Subject', 'Image']


Now that we have configured our datasets, we need to identify the dataset types so we can distinguish between them.

In [10]:
# Create a new dataset
ml_instance.add_term(vt.dataset_type, "DemoSet", description="A test dataset")
ml_instance.add_term(vt.dataset_type, 'Partitioned', description="A partitioned dataset for ML training.")
ml_instance.add_term(vt.dataset_type, "Subject", description="A test dataset")
ml_instance.add_term(vt.dataset_type, "Image", description="A test dataset")
ml_instance.add_term(vt.dataset_type, "Training", description="Training dataset")
ml_instance.add_term(vt.dataset_type, "Testing", description="Training dataset")
ml_instance.add_term(vt.dataset_type, "Validation", description="Validation dataset")

ml_instance.list_vocabulary_terms(vt.dataset_type)

[VocabularyTerm(name='DemoSet', synonyms=[], id='ml-test:38J', uri='/id/38J', description='A test dataset', rid='38J'),
 VocabularyTerm(name='Partitioned', synonyms=[], id='ml-test:38M', uri='/id/38M', description='A partitioned dataset for ML training.', rid='38M'),
 VocabularyTerm(name='Subject', synonyms=[], id='ml-test:38P', uri='/id/38P', description='A test dataset', rid='38P'),
 VocabularyTerm(name='Image', synonyms=[], id='ml-test:38R', uri='/id/38R', description='A test dataset', rid='38R'),
 VocabularyTerm(name='Training', synonyms=[], id='ml-test:38T', uri='/id/38T', description='Training dataset', rid='38T'),
 VocabularyTerm(name='Testing', synonyms=[], id='ml-test:38W', uri='/id/38W', description='Training dataset', rid='38W'),
 VocabularyTerm(name='Validation', synonyms=[], id='ml-test:38Y', uri='/id/38Y', description='Validation dataset', rid='38Y')]

Now create datasets and populate with elements from the test catalogs.

In [11]:
system_columns = ['RCT', 'RMT', 'RCB', 'RMB']

subject_dataset = ml_instance.create_dataset(['DemoSet', 'Subject'], description="A subject dataset")
image_dataset = ml_instance.create_dataset(['DemoSet', 'Image'], description="A image training dataset")
datasets = pd.DataFrame(ml_instance.find_datasets()).drop(columns=system_columns)
display(
    Markdown('## Datasets'),
    datasets)

## Datasets

,RID,Description,Dataset_Type
0,390,A subject dataset,"[DemoSet, Subject]"
1,396,A image training dataset,"[DemoSet, Image]"


And now that we have defined some datasets, we can add elements of the appropriate type to them.  We can see what is in our new datasets by listing the dataset members.

In [12]:
dp = ml_instance.domain_path  # Each call returns a new path instance, so only call once...
subject_rids = [i['RID'] for i in dp.tables['Subject'].entities().fetch()]
image_rids = [i['RID'] for i in dp.tables['Image'].entities().fetch()]

ml_instance.add_dataset_members(dataset_rid=subject_dataset, members=subject_rids)
ml_instance.add_dataset_members(dataset_rid=image_dataset, members=image_rids)

# List the contents of our datasets, and let's not include columns like modify time.
display(
    Markdown('## Subject Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(subject_dataset)['Subject']).drop(columns=system_columns),
    Markdown('## Image Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(image_dataset)['Image']).drop(columns=system_columns))

## Subject Dataset

,RID,Name
0,31A,Thing1
1,31C,Thing2
2,31E,Thing3
3,31G,Thing4
4,31J,Thing5
5,31M,Thing6
6,31P,Thing7
7,31R,Thing8
8,31T,Thing9
9,31W,Thing10


## Image Dataset

,RID,URL,Filename,Description,Length,MD5,Name,Subject
0,32J,/hatrac/Image/996a51e1ce68849b3af787250b5685d4...,test_31A.txt,A test image,32,996a51e1ce68849b3af787250b5685d4,None,31A
1,32M,/hatrac/Image/6bbfddbadb096135329062bd86a88ab1...,test_31C.txt,A test image,31,6bbfddbadb096135329062bd86a88ab1,None,31C
2,32P,/hatrac/Image/783a4326e686b55cdc5df43e546afdca...,test_31E.txt,A test image,31,783a4326e686b55cdc5df43e546afdca,None,31E
3,32R,/hatrac/Image/c3cf19a9163f247fdcd1930a8f51c6e3...,test_31G.txt,A test image,31,c3cf19a9163f247fdcd1930a8f51c6e3,None,31G
4,32T,/hatrac/Image/ee9e9acf7b5d89a5732f24a960765381...,test_31J.txt,A test image,32,ee9e9acf7b5d89a5732f24a960765381,None,31J
5,32W,/hatrac/Image/a89a059ceb86cc9039328ce63f0e4197...,test_31M.txt,A test image,31,a89a059ceb86cc9039328ce63f0e4197,None,31M
6,32Y,/hatrac/Image/58f2199c8ae244ffd420187fb0bb85db...,test_31P.txt,A test image,31,58f2199c8ae244ffd420187fb0bb85db,None,31P
7,330,/hatrac/Image/45a480dfcc9a527f6f1b40dbf64195a9...,test_31R.txt,A test image,31,45a480dfcc9a527f6f1b40dbf64195a9,None,31R
8,332,/hatrac/Image/d735918a0e7c8d4a60c9ec227edae676...,test_31T.txt,A test image,31,d735918a0e7c8d4a60c9ec227edae676,None,31T
9,334,/hatrac/Image/6dabf396e878bef914386435d201f871...,test_31W.txt,A test image,31,6dabf396e878bef914386435d201f871,None,31W


## Create partitioned dataset

Now let's create some subsets of the original dataset based on subject level metadata. We are going to create the subsets based on the metadata values of the subjects. We will download the subject dataset and look at its metadata to figure out how to partition the original data. Since we are not going to look at the images, we use download_dataset_bag, rather than materialize_bag.

In [ ]:
bag_path, bag_rid = ml_instance.download_dataset_bag(subject_dataset)
ml_instance.materialize_dataset_bag(subject_dataset)
dataset_bag = DatasetBag(bag_path)
print(f"Bag materialized to {bag_path}")

The domain model has two object: Subject and Images where an Image is associated with a subject, but a subject can have multiple images associated with it.  Let's look at the subjects and partition into test and training datasets.

In [ ]:
# Get information about the subjects.....
subject_df = dataset_bag.get_table_as_dataframe('Subject')[['RID', 'Name']]
image_df = dataset_bag.get_table_as_dataframe('Image')[['RID', 'Subject', 'URL']]
metadata_df = subject_df.join(image_df, lsuffix="_subject", rsuffix="_image")
display(metadata_df)

For ths example, lets partition the data based on the name of the subject.  Of course in real examples, we would do a more complex analysis in deciding
what subset goes into each data set.

In [ ]:
def thing_number(name: pd.Series) -> pd.Series:
    return name.map(lambda n: int(n.replace('Thing','')))

training_rids = metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 0]['RID_image'].tolist()
testing_rids =  metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 1]['RID_image'].tolist()
validation_rids = metadata_df.loc[lambda df: thing_number(df['Name']) % 3 == 2]['RID_image'].tolist()

print(f'Training images: {training_rids}')
print(f'Testing images: {testing_rids}')
print(f'Validation images: {validation_rids}')


Now that we know what we want in each dataset, lets create datasets for each of our partitioned elements along with a nested dataset to track the entire collection.

In [ ]:
nested_dataset = ml_instance.create_dataset(['Partitioned', 'Image'], description='A nested dataset for machine learning')
training_dataset = ml_instance.create_dataset('Training', description='An image dataset for training')
testing_dataset = ml_instance.create_dataset('Testing', description='A image dataset for testing')
validation_dataset = ml_instance.create_dataset('Validation', description='A image dataset for validation')
pd.DataFrame(ml_instance.find_datasets()).drop(columns=system_columns)

And then fill the datasets with the appropriate members.

In [ ]:

ml_instance.add_dataset_members(dataset_rid=nested_dataset, members=[training_dataset, testing_dataset, validation_dataset])
ml_instance.add_dataset_members(dataset_rid=training_dataset, members=training_rids)
ml_instance.add_dataset_members(dataset_rid=testing_dataset, members=testing_rids)
ml_instance.add_dataset_members(dataset_rid=validation_dataset, members=validation_rids)


Ok, lets see what we have now.

In [ ]:
display(
    Markdown('## Nested Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(nested_dataset)['Dataset']).drop(columns=system_columns),
    Markdown('## Training Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(training_dataset)['Image']).drop(columns=system_columns),
    Markdown('## Testing Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(testing_dataset)['Image']).drop(columns=system_columns),
    Markdown('## Validation Dataset'),
    pd.DataFrame(ml_instance.list_dataset_members(validation_dataset)['Image']).drop(columns=system_columns))

As our very last step, lets get a PID that will allow us to share and cite the dataset that we just created

In [ ]:
dataset_citation = ml_instance.cite(nested_dataset)
display(
    HTML(f'Nested dataset citation: <a href={nested_dataset}>{nested_dataset}</a>')
)

In [ ]:
display(HTML(f'<a href={ml_instance.chaise_url("Dataset")}>Browse Datasets</a>'))

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)